In [4]:
import numpy as np
import pandas as pd
import pulp as lp
import pickle as pkl
import time
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime as dt
import ALMPlanner as ALM
import ALMChart as ALMc
import os
from ALMChart import standardized_chart
from ipywidgets import widgets, interact_manual, GridBox, Layout

path_example = os.path.join(os.getcwd(), "example")

buyandhold_portfolios = ALM.load_scenario("scenario","eff_front")
user_portfolio_code = {
    0:23,
    1:37,
    2:56,
    3:74,
    4:84
    }

In [10]:
######
EX = 1
######
user_risk_profile = 2

if EX == 1:
    problem = ALM.ALMplanner(start = "2021", end = "2041", user_risk_profile = user_portfolio_code[user_risk_profile])
    problem.liabilities.insert("car", "2026", 25000, 25000*0.65)
    problem.liabilities.insert("university", "2029", 50000, 50000*0.95)
    problem.liabilities.insert("hawaii", "2037",25000, 25000*0.85) 
    problem.assets.insert("init","Jan 2021",30000)
    ALM.add_recurrent(problem, start = "Jan 2022", end = "Jan 2027", type = "asset", value = 10000, label = "ass")
elif EX == 2:
    problem = ALM.ALMplanner(start = "Jan 2021", end = "Jan 2061", user_risk_profile = user_portfolio_code[user_risk_profile])
    ALM.add_recurrent(problem, start = "Jan 2021", end = "Jan 2040", type = "asset", value = 1000, label = "ass")
    ALM.add_recurrent(problem, start = "Jan 2041", end = "Jan 2060", type = "goal", value_tg = 1500, value_lb = 1100, label = "ret")
elif EX == 3:
    problem = ALM.ALMplanner(start = "2021", end = "2041", user_risk_profile = user_portfolio_code[user_risk_profile])
    problem.liabilities.insert("car", "2036", 45000, 45000*0.65) 
    problem.assets.insert("init","Jan 2021",40000)
elif EX == 4:
    problem = ALM.ALMplanner(start = "2021", end = "2041", user_risk_profile = user_portfolio_code[user_risk_profile])
    problem.liabilities.insert("car", "2026", 30000, 30000*0.65)
    problem.liabilities.insert("university", "2029", 50000, 50000*0.95)
    problem.liabilities.insert("hawaii", "2037",30000, 30000*0.85) 
    problem.assets.insert("init","Jan 2021",30000)
    ALM.add_recurrent(problem, start = "Jan 2022", end = "Jan 2027", type = "asset", value = 10000, label = "ass")

filename = os.path.join(path_example, "example"+str(EX)+".pkl")
fileopen = open(filename, "wb")
pkl.dump(problem, fileopen)
fileopen.close()


In [3]:
ALMc.display(problem,bar_width = 6)

In [4]:
problem.check_feasibility()
problem.get_feasibility()

check feasibility ended in 12.04 s with Optimal solution
Feasible problem


In [5]:
ALMc.display(problem,bar_width = 6)

In [6]:
GB_model = ALM.ALMGoalBased(problem)
GB_model.solve()

BaH_model = ALM.ALMBuyAndHold_2(problem)
BaH_model.solve()

GoalBased model generated in 13.99 s
Solve ended in 1.41 s with Optimal solution
Solve ended in 9.34 s with


# Display solution

In [7]:
sol = GB_model.solution
sol_bah = BaH_model.solution


In [8]:
colormap_liab = {}
colormap_ETF = {}
Ltot = list(problem.liabilities.set)
Ltot.append("extra_wealth")
it = -1
for e in Ltot:
    it = it+1
    colormap_liab[e] = px.colors.qualitative.Plotly[it%10]

it=-1
for p in problem.P:
    it = it+1
    colormap_ETF[p] = px.colors.qualitative.Plotly[it%10]
it = -1

#colormap_ETF = {problem.P[0]:"royalblue", problem.P[1]:"green", problem.P[2]:"lime", problem.P[3]:"yellow", problem.P[4]:"orange", problem.P[5]:"red"}

In [9]:
plt = ALMc.AssetSplitDetailsChart(problem, sol, "Goal", colormap_liab)
#plt.show()
plt = ALMc.AssetSplitDetailsChart(problem, sol_bah, "Goal", colormap_liab)
#plt.show()

In [10]:
plt = ALMc.AssetSplitDetailsChart(problem, sol, "ETF", colormap_ETF)    
#plt.show()
plt = ALMc.AssetSplitDetailsChart(problem, sol_bah, "ETF", colormap_ETF)
#plt.show()

In [48]:
n_scen = None
perc = False
showlegend = True
horizontal_spacing = 0.03
current_user_risk_profile = problem.user_risk_profile

fig1 = go.FigureWidget(make_subplots(rows = 1, cols = 2, shared_yaxes= True, horizontal_spacing = horizontal_spacing))
plt = ALMc.AssetAllocationChart(problem,sol,n_scen=n_scen, perc = perc)
#fig1 = go.FigureWidget(plt)
fig1.add_traces(data=plt.data, rows = 1, cols = 1)
plt = ALMc.AssetAllocationChart(problem,sol_bah,n_scen=n_scen, perc = perc)#, showlegend= False)
#fig1_bah = go.FigureWidget(plt)
fig1.add_traces(data=plt.data, rows = 1, cols = 2)

ALMc.standardized_chart(fig1, perc = perc, showlegend= showlegend)
ALMc.standardized_chart(fig1_bah, perc = perc, showlegend= showlegend)

asset = ALMc.AssetSplitDetailsChart(problem, sol, "ETF", colormap_ETF)    
asset_bah = ALMc.AssetSplitDetailsChart(problem, sol_bah, "ETF", colormap_ETF)

prob, avg = ALMc.GoalRiskDetails(problem, sol, perc)
prob_bah, avg_bah = ALMc.GoalRiskDetails(problem, sol_bah, perc, showlegend = False)

fig2 = go.FigureWidget(make_subplots(rows = 1, cols = 2, shared_yaxes= True, horizontal_spacing = horizontal_spacing))
fig2.add_traces(data=prob.data, rows = 1, cols = 1)
fig2.add_traces(data=prob_bah.data, rows = 1, cols = 2)
fig2.update_layout(barmode = "stack")
fig2 = ALMc.standardized_chart(fig2, perc= True, showlegend=showlegend)

fig3 = go.FigureWidget(make_subplots(rows = 1, cols = 2, shared_yaxes= True, horizontal_spacing = horizontal_spacing))
fig3.add_traces(data=avg.data, rows = 1, cols = 1)
fig3.add_traces(data=avg_bah.data, rows = 1, cols = 2)
fig3.update_layout(barmode = "overlay")
fig3 = ALMc.standardized_chart(fig3, perc = perc, showlegend=showlegend)

success_prob, fail_prob, tot_wealth_avg, paid_advance_avg, avg_loss_in_failure = ALMc.EoPWealthInfo(problem, sol)
success_prob_bah, fail_prob_bah, tot_wealth_avg_bah, paid_advance_avg_bah, avg_loss_in_failure_bah = ALMc.EoPWealthInfo(problem, sol_bah)

plan_success_prob = widgets.HTML(f"<br />Plan success/failure probability:<br />   Success: {success_prob}%,<br />   Failure: {fail_prob}%<br />")
eop_wealth_summary = widgets.HTML(f"<br />End-of-period wealth summary:<br />                expected wealth: {int(tot_wealth_avg)}€<br />        thereof paid in advance: {int(paid_advance_avg)}€<br />   shortfall in case of failure: {int(avg_loss_in_failure)}€")

plan_success_prob_bah = widgets.HTML(f"<br />Plan success/failure probability:<br />   Success: {success_prob_bah}%,<br />   Failure: {fail_prob_bah}%<br />")
eop_wealth_summary_bah = widgets.HTML(f"<br />End-of-period wealth summary:<br />                expected wealth: {int(tot_wealth_avg_bah)}€<br />        thereof paid in advance: {int(paid_advance_avg_bah)}€<br />   shortfall in case of failure: {int(avg_loss_in_failure_bah)}€")

goal_based_summary = widgets.HTML(value='<b><h1><p style="text-align:center">Goal-based strategy</p></h1></b>', layout=widgets.Layout(display='flex', justify_content='center'))
BaH_summary = widgets.HTML(value='<b><h1><p style="text-align:center">Buy-and-hold strategy</p></h1></b>', layout=widgets.Layout(display='flex', justify_content='center'))

goal_based_summary2 = widgets.HTML(value='<b><h1><p style="text-align:center">Goal-based strategy</p></h1></b>', layout=widgets.Layout(display='flex', justify_content='center'))
BaH_summary2 = widgets.HTML(value='<b><h1><p style="text-align:center">Buy-and-hold strategy</p></h1></b>', layout=widgets.Layout(display='flex', justify_content='center'))



dashboard = GridBox(children=[goal_based_summary,BaH_summary,fig1,plan_success_prob,fig2,plan_success_prob_bah,eop_wealth_summary,fig3,eop_wealth_summary_bah],
        layout=Layout(
            width='80%',
            grid_template_rows='10% auto auto auto',
            grid_template_columns='25% 25% 25% 25%',
            grid_template_areas='''
            "goal_based_summary goal_based_summary BaH_summary BaH_summary" 
            "fig1 goal_based_summary2 fig1_bah BaH_summary2" 
            "plan_success_prob fig2 fig2 plan_success_prob_bah" 
            "eop_wealth_summary fig3 fig3 eop_wealth_summary_bah"''')
       )
print(dashboard.layout)

Layout(grid_template_areas='\n            "goal_based_summary goal_based_summary BaH_summary BaH_summary" \n            "fig1 goal_based_summary2 fig1_bah BaH_summary2" \n            "plan_success_prob fig2 fig2 plan_success_prob_bah" \n            "eop_wealth_summary fig3 fig3 eop_wealth_summary_bah"', grid_template_columns='25% 25% 25% 25%', grid_template_rows='10% auto auto auto', width='80%')


In [34]:
@interact_manual(percentual = perc, BaH_strategy = widgets.IntSlider(range = [0,(len(buyandhold_portfolios)-1)], value = problem.user_risk_profile, style = {'description_width': 'initial'} ))
def update(percentual,BaH_strategy):
    global current_user_risk_profile
    global sol
    global sol_bah
    perc = percentual
    ## Reset Chart
    with fig1.batch_update():
        n_index = int(len(fig1.data)/2)
        for i in np.arange(n_index):
            fig1.data[i].y = np.zeros(len(fig1.data[i].y))
            fig1.data[i+n_index].y = np.zeros(len(fig1.data[i+n_index].y))
        ALMc.standardized_chart(fig1, perc = perc, showlegend= True)
    with fig2.batch_update():
        n_index = int(len(fig2.data)/2)
        for i in np.arange(n_index):
            fig2.data[i].y = np.zeros(len(fig2.data[i].y))
            fig2.data[i+n_index].y = np.zeros(len(fig2.data[i+n_index].y))
        ALMc.standardized_chart(fig2, perc = True, showlegend= True)
    with fig3.batch_update():
        n_index = int(len(fig3.data)/2)
        for i in np.arange(n_index):
            fig3.data[i].y = np.zeros(len(fig3.data[i].y))
            fig3.data[i+n_index].y = np.zeros(len(fig3.data[i+n_index].y))
        ALMc.standardized_chart(fig3, perc = perc, showlegend= True)
    if not(current_user_risk_profile == BaH_strategy):
        current_user_risk_profile = BaH_strategy
        sol.update_end(problem,buyandhold_portfolios[BaH_strategy])
        BaH_model2 = ALM.ALMBuyAndHold_2(problem)
        BaH_model2.solve(buyandhold_portfolios[BaH_strategy])
        sol_bah = BaH_model2.solution
    ## Update data with new input
    plt1 = ALMc.AssetAllocationChart(problem,sol,n_scen=n_scen, perc = perc, portfolio_strategy = buyandhold_portfolios[BaH_strategy])
    plt2 = ALMc.AssetAllocationChart(problem,sol_bah,n_scen=n_scen, perc = perc, showlegend= False, portfolio_strategy = buyandhold_portfolios[BaH_strategy])
    prob, avg = ALMc.GoalRiskDetails(problem, sol, perc)
    prob_bah, avg_bah = ALMc.GoalRiskDetails(problem, sol_bah, perc, showlegend = False)
    success_prob, fail_prob, tot_wealth_avg, paid_advance_avg, avg_loss_in_failure = ALMc.EoPWealthInfo(problem, sol)
    success_prob_bah, fail_prob_bah, tot_wealth_avg_bah, paid_advance_avg_bah, avg_loss_in_failure_bah = ALMc.EoPWealthInfo(problem, sol_bah)
    ## update chart with new output
    with fig1.batch_update():
        n_index = int(len(fig1.data)/2)
        for i in np.arange(n_index):
            fig1.data[i].y = plt1.data[i].y
            fig1.data[i+n_index].y = plt2.data[i].y
        ALMc.standardized_chart(fig1, perc = perc, showlegend= True)
    with fig2.batch_update():
        n_index = int(len(fig2.data)/2)
        for i in np.arange(n_index):
            fig2.data[i].y = prob.data[i].y
            fig2.data[i+n_index].y = prob_bah.data[i].y
        ALMc.standardized_chart(fig2, perc = True, showlegend= True)
    with fig3.batch_update():
        n_index = int(len(fig3.data)/2)
        for i in np.arange(n_index):
            fig3.data[i].y = avg.data[i].y
            fig3.data[i+n_index].y = avg_bah.data[i].y
        ALMc.standardized_chart(fig3, perc = perc, showlegend= True)
    return


display(dashboard)


interactive(children=(Checkbox(value=False, description='percentual'), IntSlider(value=56, description='BaH_st…

GridBox(children=(HTML(value='<b><h1><p style="text-align:center">Goal-based strategy</p></h1></b>', layout=La…

In [49]:
A = widgets.HTML(value='<b><p style="text-align:center">A html</p></b>', layout=widgets.Layout(justify_content='center'))
B = widgets.HTML(value='<b><p style="text-align:center">B html</p></b>', layout=widgets.Layout(justify_content='center'))

C = widgets.HTML(value='<b><p style="text-align:center">C html</p></b>', layout=widgets.Layout(justify_content='center'))
D = widgets.HTML(value='<b><p style="text-align:center">D html</p></b>', layout=widgets.Layout(justify_content='center'))


GridBox(children=[A,B,C,D],
        layout=Layout(
            width='50%',
            grid_template_rows='10% 45% 45%',
            grid_template_columns='25% 25% 25% 25%',
            grid_template_areas='''
            "A A B B" 
            "A A C C" 
            "D D C C" 
            ''')
        )

GridBox(children=(HTML(value='<b><p style="text-align:center">A html</p></b>', layout=Layout(justify_content='…

In [ ]:
def AssetSplitDetailsChart(planner, solution, groupby, colormap):
    P = planner.P
    A = planner.assets.set
    L = planner.liabilities.set
    Assets_split = pd.DataFrame(index = np.arange(len(P)*len(A)*(len(L)+1)), columns = ["Asset", "Goal", "ETF", "Value"])
    it= -1
    for a in A:
        for p in P:
            for l in L:
                it = it+1
                Assets_split["Asset"][it] = a
                Assets_split["Goal"][it] = l
                Assets_split["ETF"][it] = p
                Assets_split["Value"][it] = solution.asset_to_goal[a][l][p]
            it = it+1
            Assets_split["Asset"][it] = a
            Assets_split["Goal"][it] = "extra_wealth"
            Assets_split["ETF"][it] = p
            Assets_split["Value"][it] = solution.asset_to_exwealth[a][p]
    AssetGroupedBy = Assets_split[["Asset", groupby, "Value"]].groupby(by=[groupby, "Asset"]).sum().reset_index()
    AssetPivot = AssetGroupedBy.pivot(index = "Asset", columns=groupby, values = "Value")
    AssetPivot["Period"] = pd.Series(planner.assets.period)
    AssetPivot = AssetPivot.sort_values(by = "Period")
    data = []
    if groupby == "Goal":
        groupby_set = list(L)
        groupby_set.append("extra_wealth")
    elif groupby == "ETF":
        groupby_set = P
    else:
        return
    data = []
    for e in groupby_set:
        data.append(go.Bar(x = AssetPivot.index, y = AssetPivot[e], marker_color = colormap[e], name = e))
    ASDChart = go.Figure(data = data, layout = go.Layout(barmode = "stack"))
    ASDChart = standardized_chart(ASDChart)
    return ASDChart